# Imports

In [ ]:
# Import utils
import numpy as np
import pandas as pd
import copy
import time
import datetime as dt
import pickle
import json
from pathlib import Path
import joblib
from joblib import dump, load, Parallel, delayed
import os
import itertools
import contextlib
from tqdm import tqdm

# Import Weights Model
import WeightsModel3
from WeightsModel3 import PreProcessing
from WeightsModel3 import RandomForestWeightsModel


# Import (Rolling Horizon) Weighted SAA models
from WeightedSAA6 import WeightedSAA
from WeightedSAA6 import RobustWeightedSAA
from WeightedSAA6 import RollingHorizonOptimization

# General paths and parameters

In [ ]:
# Set folder names as global variables
os.chdir('/home/fesc/DataDrivenDynamicInventoryControl/')
global PATH_DATA, PATH_WEIGHTSMODEL, PATH_RESULTS

PATH_DATA = '/home/fesc/DataDrivenDynamicInventoryControl/Data' 
PATH_WEIGHTSMODEL = '/home/fesc/DataDrivenDynamicInventoryControl/Data/WeightsModel'
PATH_RESULTS = '/home/fesc/DataDrivenDynamicInventoryControl/Data/Results'

In [ ]:
# Time period and SKU ranges
T = 13                  # Planning horizon T
ts = range(1,13+1)      # Periods t=1,...,T of the planning horizon
taus = range(0,4+1)     # Look-aheads tau=0,...,4 to use
es = [1,3,6,9,12]       # Uncertainty set specifications e=1,...,12
SKUs = range(1,460+1)   # Products (SKUs) k=1,...,M

# Train/test split (first timePeriods of testing horizon)
test_start = 114

# Cost param settings
cost_params = [

    {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
    {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
    {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

]

# Training and Sampling

## Global Training and Samping

The two global models (using 'Global Training and Sampling') are **Rolling Horizon Global Weighted SAA (GwSAA)**, which is our model, and **Rolling Horizon Global Robust Weighted SAA (GwSAA-R)**, which is the analogous model with robust extension.

Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'global' distribution $\{\{w_{j,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$, with weight functions $w_{j,t,\tau}(\,\cdot\,)$ trained (once for all products) on data $S_{t,\tau}^{\,\text{Global}}=\{\{(x_{j,t}^{\,i},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$.

### Data Preprocessing

We first load and pre-process the data. This includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ where n is the number of training observations (rows) in the data
- **Y_Data** (pd.DataFrame) is the demand data $d_{j,n}$ (a times series per product)
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_global_not_reshaped'
weightsmodel_name = 'rfwm_global_not_reshaped'

In [ ]:
# Read data
ID_Data = pd.read_csv(PATH_DATA+'/ID_Data.csv')
X_Data = pd.read_csv(PATH_DATA+'/X_Data.csv')
X_Data_Columns = pd.read_csv(PATH_DATA+'/X_Data_Columns.csv')
Y_Data = pd.read_csv(PATH_DATA+'/Y_Data.csv')

In [ ]:
# Select features
X_Data_Columns = X_Data_Columns.loc[X_Data_Columns.Global == 'YES']
X_Data = X_Data[X_Data_Columns.Feature.values]

In [ ]:
# Ensure data is sorted by SKU and sale_yearweek for preprocessing
data = pd.concat([ID_Data, X_Data, Y_Data], axis=1).sort_values(by=['SKU', 'sale_yearweek']).reset_index(drop=True)
ID_Data = data[ID_Data.columns]
X_Data = data[X_Data.columns]
Y_Data = data[Y_Data.columns]

# Create multi-period demand vectors
data = pd.concat([ID_Data, Y_Data], axis=1)
Y = {}
for tau in taus:
    Y['Y'+str(tau)] = data.groupby(['SKU']).shift(-tau)['Y']
Y_Data = pd.DataFrame(Y)

### Weights model

The weights models - and thus the data used, weight functions, and weights per sample - are the same for the two global models **GwSAA** and **GwSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor). Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$).

#### Tune weights model

To tune the hyper parameters of the global random forest weights model, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Set parameters
model_params = {
    'oob_score': True,
    'random_state': 12345,
    'n_jobs': 4,
    'verbose': 0
}

hyper_params_grid = {
    'n_estimators': [1000],
    'max_depth': [None],
    'min_samples_split': [x for x in range(20, 1000, 20)],  
    'min_samples_leaf': [x for x in range(10, 1000, 10)],  
    'max_features': [x for x in range(8, 256, 8)],   
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'bootstrap': [True],
    'max_samples': [0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
}    


tuning_params = {     
    'n_iter': 100,
    'scoring': {'MSE': 'neg_mean_squared_error'},
    'return_train_score': True,
    'refit': 'MSE',
    'random_state': 12345,
    'n_jobs': 8,
    'verbose': 2
}    

random_search = True
print_status = True

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Initialize preprocessing module
    pp = PreProcessing()
        
    # Select and training data
    args = {'train': (ID_Data.sale_yearweek < test_start - tau)}
    
    id_train = pp.train_test_split2(ID_Data, **args)
    X_train = pp.train_test_split2(X_Data, **args, to_array=True)
    y_train = pp.train_test_split2(Y_Data, **args, rolling_horizon=[l for l in range(0,tau+1)], to_array=True)
   
    # Initialize
    weightsmodel = RandomForestWeightsModel(model_params)

    # CV search
    cv_folds = pp.split_timeseries_cv(n_splits=3, timePeriods=id_train.sale_yearweek)
    cv_results = weightsmodel.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status)
    weightsmodel.save_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau)+'.joblib')

#### Fit weight functions and generate weights

We now fit the global random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$ and over periods $t=1,...,T$. This is done across all products at once (global training). Then, for each $\tau=0,...,4$ and over periods $t=1,...,T$, we generate for each product (SKU) $k=1,...,M$ the weights given the test feature $x_{k,t}$. This is done *jointly* across products (by using $x_{t}=(x_{1,t},...,x_{M,t})^{\top}$) for computational efficiency - the weights for each individual product can be extracted afterwards.

In [ ]:
# Set parameters
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

print_status = True

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Initialize
    samples, weightfunctions, weightfunctions_times, weights, weights_times = {}, {}, {}, {}, {}
        
    # For each period t=1,...,T
    for t in ts:

        # Adjust look-ahead tau to account for end of horizon
        tau_ = min(tau,T-t)
        
        # Status
        print('#### Look-ahead tau='+str(tau)+' (tau\'='+str(tau_)+'), period t='+str(t)+'...')
        start_time = dt.datetime.now().replace(microsecond=0)
                
        # Generate samples, fit weight functions, and generate weights (based on tuned weights model)
        weightsmodel = RandomForestWeightsModel()
        weightsmodel.load_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau_)+'.joblib')
        
        res = weightsmodel.training_and_sampling2(ID_Data, X_Data, Y_Data, tau=tau_, timePeriods=ID_Data.sale_yearweek,
                                                  timePeriodsTestStart=test_start+t-1, model_params=model_params)
        samples[t], weightfunctions[t], weightfunctions_times[t], weights[t], weights_times[t] = res
        
        # Status
        print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    
        
    # Save
    _ = joblib.dump(samples, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')  
    _ = joblib.dump(weightfunctions, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weightfunctions_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_times_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_times_tau'+str(tau)+'.joblib')    

In [ ]:
##### Testing with "old" hyper params for global RF models ...

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_global_not_reshaped_old_rf_params'
weightsmodel_name = 'rfwm_global_not_reshaped_old_rf_params'

In [ ]:
rfwm_0 = joblib.load('/home/fesc/DataDrivenDynamicInventoryControl/rf_mv_t1.joblib')
rfwm_1 = joblib.load('/home/fesc/DataDrivenDynamicInventoryControl/rf_mv_t2.joblib')
rfwm_2 = joblib.load('/home/fesc/DataDrivenDynamicInventoryControl/rf_mv_t3.joblib')
rfwm_3 = joblib.load('/home/fesc/DataDrivenDynamicInventoryControl/rf_mv_t4.joblib')
rfwm_4 = joblib.load('/home/fesc/DataDrivenDynamicInventoryControl/rf_mv_t5.joblib')

In [ ]:
# Set parameters
model_params = {
    'n_jobs': 96,
    'verbose': 0
}

print_status = True

In [ ]:
hyper_params = {}


hyper_params[0] = {
    
    'max_features': 144, 
    'min_samples_leaf': 10,
    'n_estimators': 500,
}

hyper_params[1] = {
    
    'max_features': 160, 
    'min_samples_leaf': 10,
    'n_estimators': 500,
}

hyper_params[2] = {
    
    'max_features': 96, 
    'min_samples_leaf': 10,
    'n_estimators': 500,
}

hyper_params[3] = {
    
    'max_features': 112, 
    'min_samples_leaf': 10,
    'n_estimators': 500,
}

hyper_params[4] = {
    
    'max_features': 48, 
    'min_samples_leaf': 10,
    'n_estimators': 500,
}

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Initialize
    samples, weightfunctions, weightfunctions_times, weights, weights_times = {}, {}, {}, {}, {}
        
    # For each period t=1,...,T
    for t in ts:

        # Adjust look-ahead tau to account for end of horizon
        tau_ = min(tau,T-t)
        
        # Status
        print('#### Look-ahead tau='+str(tau)+' (tau\'='+str(tau_)+'), period t='+str(t)+'...')
        start_time = dt.datetime.now().replace(microsecond=0)
                
        # Generate samples, fit weight functions, and generate weights (based on tuned weights model)
        weightsmodel = RandomForestWeightsModel(hyper_params=hyper_params[tau_])
        
        res = weightsmodel.training_and_sampling2(ID_Data, X_Data, Y_Data, tau=tau_, timePeriods=ID_Data.sale_yearweek,
                                                  timePeriodsTestStart=test_start+t-1, model_params=model_params)
        samples[t], weightfunctions[t], weightfunctions_times[t], weights[t], weights_times[t] = res
        
        # Status
        print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    
        
    # Save
    _ = joblib.dump(samples, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')  
    _ = joblib.dump(weightfunctions, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weightfunctions_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_times_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_times_tau'+str(tau)+'.joblib')    

In [ ]:
## Why does it take longer for t=5 (tau=0)?? Check for other tau's... is there a data issue?

# tau=0, t=5
# tau=1, t=6
# tau=2, t=7
# tau=3, t=8
# tau=4, t=9

# ... what is happening here??

# it happends when t-tau == 5

In [ ]:
25*5

## Local Training and Sampling

The two local models (using 'Local Training and Sampling') are **Rolling Horizon Local Weighted SAA (wSAA)**, and **Rolling Horizon Local Robust Weighted SAA (wSAA-R)**, which is the analogous model with robust extension.

Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'local' distribution $\{w_{k,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$, with weight functions $w_{k,t,\tau}(\,\cdot\,)$ trained on data $S_{k,t,\tau}^{\,\text{Local}}=\{(x_{k,t}^{\,i},d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$ for each product $k=1,...,M$ separately.

### Data Preprocessing

We first load and pre-process the data. This includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ where n is the number of training observations (rows) in the data
- **Y_Data** (pd.DataFrame) is the demand data $d_{j,n}$ (a times series per product)
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_local_not_reshaped'
weightsmodel_name = 'rfwm_local_not_reshaped'

In [ ]:
# Read data
ID_Data = pd.read_csv(PATH_DATA+'/ID_Data.csv')
X_Data = pd.read_csv(PATH_DATA+'/X_Data.csv')
X_Data_Columns = pd.read_csv(PATH_DATA+'/X_Data_Columns.csv')
Y_Data = pd.read_csv(PATH_DATA+'/Y_Data.csv')

In [ ]:
# Select features
X_Data_Columns = X_Data_Columns.loc[X_Data_Columns.Local == 'YES']
X_Data = X_Data[X_Data_Columns.Feature.values]

In [ ]:
# Ensure data is sorted by SKU and sale_yearweek for preprocessing
data = pd.concat([ID_Data, X_Data, Y_Data], axis=1).sort_values(by=['SKU', 'sale_yearweek']).reset_index(drop=True)

ID_Data = data[ID_Data.columns]
X_Data = data[X_Data.columns]
Y_Data = data[Y_Data.columns]

In [ ]:
# Create multi-period demand vectors
data = pd.concat([ID_Data, Y_Data], axis=1)
Y = {}
for tau in taus:
    Y['Y'+str(tau)] = data.groupby(['SKU']).shift(-tau)['Y']
Y_Data = pd.DataFrame(Y)

### Weights model

The weights model - and thus the data used, weight functions, and weights per sample - are the same for the two local models **wSAA** and **wSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor) and for each product (SKU) $k=1,...,M$ separately. Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) for each product (SKU) $k=1,...,M$ separately and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$ separatey).

#### Tune weights model

To tune the hyper parameters of the local random forest weights model for each product (SKU) $k=1,...,M$, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Set parameters to tune random forest weights kernels
model_params = {
    'oob_score': True,
    'random_state': 12345,
    'n_jobs': 1,
    'verbose': 0
}

hyper_params_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None],
    'min_samples_split': [x for x in range(2, 20, 1)],  
    'min_samples_leaf': [x for x in range(2, 10, 1)],  
    'max_features': [x for x in range(8, 256, 8)],   
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'bootstrap': [True],
    'max_samples': [0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
}    


tuning_params = {     
    'n_iter': 100,
    'scoring': {'MSE': 'neg_mean_squared_error'},
    'return_train_score': True,
    'refit': 'MSE',
    'random_state': 12345,
    'n_jobs': 32,
    'verbose': 0
}    

random_search = True
print_status = False

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Initialize
    cv_results = {}
    
    # For each product (SKU) k=1,...,M
    for SKU in SKUs:

        # Initialize preprocessing module
        pp = PreProcessing()

        # Select and reshape training and test data
        args = {'train': (ID_Data.SKU == SKU) & (ID_Data.sale_yearweek < test_start - tau)}

        id_train = pp.train_test_split2(ID_Data, **args)
        X_train = pp.train_test_split2(X_Data, **args, to_array=True)
        y_train = pp.train_test_split2(Y_Data, **args, rolling_horizon=[l for l in range(0,tau+1)], to_array=True)

        # Initialize
        weightsmodel = RandomForestWeightsModel(model_params)

        # CV search
        cv_folds = pp.split_timeseries_cv(n_splits=3, timePeriods=id_train.sale_yearweek)
        cv_results[SKU] = weightsmodel.tune(X_train, y_train, cv_folds, hyper_params_grid, 
                                            tuning_params, random_search, print_status)
        
        # Status
        print('SKU '+str(SKU)+' of '+str(len(SKUs))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True)

    # Save
    _ = joblib.dump(cv_results, PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau)+'.joblib')
    print('')

#### Fit weight functions and generate weights

We now fit a local random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately (local training). Then, for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately, we generate the weights given the test feature $x_{k,t}$. This is done *separately* for each product (SKU) $k=1,...,M$.

In [ ]:
# Set parameters
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

print_status = True

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Initialize
    samples, weightfunctions, weightfunctions_times, weights, weights_times = {}, {}, {}, {}, {}
    
    # For each product (SKU) k=1,...,M
    for SKU in SKUs:
        
        # Initialize
        samples[SKU], weightfunctions[SKU], weightfunctions_times[SKU], weights[SKU], weights_times[SKU] = {}, {}, {}, {}, {}
        
        # For each period t=1,...,T
        for t in ts:
        
            # Adjust look-ahead tau to account for end of horizon
            tau_ = min(tau,T-t)

            # Generate samples, fit weight functions, and generate weights (based on tuned weights model)
            weightsmodel = RandomForestWeightsModel()
            weightsmodel.load_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau_)+'.joblib', SKU=SKU)
            res = weightsmodel.training_and_sampling2(ID_Data.loc[ID_Data.SKU==SKU], X_Data.loc[ID_Data.SKU==SKU], Y_Data.loc[ID_Data.SKU==SKU], 
                                                      tau=tau_, timePeriods=ID_Data.loc[ID_Data.SKU==SKU].sale_yearweek, 
                                                      timePeriodsTestStart=test_start+t-1, model_params=model_params)
            samples[SKU][t], weightfunctions[SKU][t], weightfunctions_times[SKU][t], weights[SKU][t], weights_times[SKU][t] = res

        # Status
        print('SKU '+str(SKU)+' of '+str(len(SKUs))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True)
        
    # Save
    _ = joblib.dump(samples, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')  
    _ = joblib.dump(weightfunctions, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weightfunctions_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_times_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_times_tau'+str(tau)+'.joblib')
    print('')

# Rolling Horizon Optimization

The code below runs an experiment for all given products (SKUs) $k=1,...,M$ over a test planning horizon $t=1,...,T$ with $T=13$ for three different cost parameter settings $\{K, u, h, b\}$ that vary the critical ratio ($CR=\frac{b}{b+h}$) of holding and backlogging yielding
- $CR=0.50$: $\{K=100, u=0.5, h=1, b=1\}$
- $CR=0.75$: $\{K=100, u=0.5, h=1, b=3\}$
- $CR=0.90$: $\{K=100, u=0.5, h=1, b=9\}$

We run experiments for different choices of the look-ahead $\tau=0,...,4$.

## Experiment functions

We first define several functions for experiments over different choices for the look-ahead $\tau=0,...,4$, cost parameter settings $\{K,u,h,b\}$, products (SKUs) $k=1,...,M$, and periods $t=1,...,T$ of the planning horizon.

In [ ]:
### Function to extract global historical demand samples, weights, and actuals for each product and test period
def prep_samples_and_weights(samples, weights=None, e=None, **kwargs):

    """
    ...
    
    """
    
    SKUs = kwargs.get('SKUs', range(1,460+1))
    ts = kwargs.get('ts', range(1,13+1))
    
    # Local
    if len(samples) == len(SKUs):
        
        # Samples
        samples_ = {}
        for SKU in SKUs:
            samples_[SKU] = {}
            for t in ts:
                samples_[SKU][t] = samples[SKU][t]['y_train']
                
        # Actuals
        actuals_ = {}
        for SKU in SKUs:
            actuals_[SKU] = {}
            for t in ts:
                actuals_[SKU][t] = samples[SKU][t]['y_test'].flatten()
                
        # Weights   
        if not weights is None:
            weights_ = {}
            for SKU in SKUs:
                weights_[SKU] = {}
                for t in ts:
                    weights_[SKU][t] = weights[SKU][t].flatten()
                    
        # Epsilons
        if not e is None:
            epsilons_ = {}
            for SKU in SKUs:
                epsilons_[SKU] = {}
                for t in ts:
                    epsilons_[SKU][t] = e*np.std(samples[SKU][t]['y_train'], axis=0).flatten()[0]
                 
    # Global
    else:
        
        # Samples
        samples_ = {}
        for t in ts:
            samples_[t] = samples[t]['y_train']

        # Actuals
        actuals_ = {}
        for SKU in SKUs:
            actuals_[SKU] = {}
            for t in ts:
                actuals_[SKU][t] = samples[t]['y_test'][samples[t]['id_test'].SKU==SKU].flatten()

        # Weights   
        if not weights is None:
            weights_ = {}
            for SKU in SKUs:
                weights_[SKU] = {}
                for t in ts:
                    weights_[SKU][t] = weights[t][samples[t]['id_test'].SKU==SKU].flatten()
                    
        # Epsilons
        if not e is None:
            epsilons_ = {}
            for SKU in SKUs:
                epsilons_[SKU] = {}
                for t in ts:
                    epsilons_[SKU][t] = e*np.std(samples[t]['y_train'][samples[t]['id_train'].SKU == SKU], axis=0).flatten()[0]

    # Return
    if not weights is None:
        if not e is None:
            return samples_, actuals_, weights_, epsilons_
        else:
            return samples_, actuals_, weights_
    else:
        if not e is None:
            return samples_, actuals_, epsilons_
        else:
            return samples_, actuals_

In [ ]:
### Function to run an experiment over a list of given cost parameter settings and the specified model
def run_experiment(wsaamodel, cost_params, actuals, samples=None, weights=None, epsilons=None, print_progress=False,
                   path_to_save=None, name_to_save=None, return_results=True, **kwargs):
    
    """
    ...
    
    """
    
    # Raise error if cost_params is not a list of dict(s)
    if not type(cost_params)==list:
        raise ValueError('Argument cost_params has to be a list of at least one dict with keys {K, u, h, b}')  
    
    # Timer
    st_exec, st_cpu = time.time(), time.process_time()

    # Status
    if print_progress and 'SKU' in kwargs: print('SKU:', kwargs['SKU'])
    
    # Initialize
    ropt, results = RollingHorizonOptimization(), pd.DataFrame()

    # For each cost param setting
    for cost_params_ in cost_params:

        # Print progress
        if print_progress: print('...cost param setting:', cost_params_)
        
        # Check if samples provided
        if not samples is None:
            
            # Apply (Weighted) SAA  model
            wsaamodel.set_params(**{**kwargs, **cost_params_})
            result = ropt.run(wsaamodel, samples, actuals, weights, epsilons)
             
            # Get T
            T = len(samples)
            
        else:
            
            # Apply ex-post clairvoyant model
            wsaamodel.set_params(**{**kwargs, **cost_params_})
            result = ropt.run_expost(wsaamodel, actuals)
            
            # Get T
            T = actuals.shape[1]

        # Store results
        meta = pd.DataFrame({'CR': cost_params_['CR'], **kwargs}, index=list(range(T)))
        results = pd.concat([results, pd.concat([meta, result], axis=1)], axis=0)

    # Save result as csv file
    if not path_to_save is None and not name_to_save is None:
        results.to_csv(path_or_buf=(path_to_save+'/'+name_to_save+'_SKU'+str(kwargs.get('SKU', None))+
                                    '_tau'+str(kwargs.get('tau', None))+'.csv'), sep=',', index=False)

    # Timer
    exec_time_sec, cpu_time_sec = time.time() - st_exec, time.process_time() - st_cpu
    
    # Status
    if print_progress: print('>>>> Done:', str(np.around(exec_time_sec/60,1)), 'minutes')

    # Return  
    return results if return_results else {'SKU': kwargs.get('SKU', None), 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}

In [ ]:
### Context manager (Credits: 'https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution')
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

## (a) Rolling Horizon Global Weighted SAA (GwSAA)

...

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_global_not_reshaped'
weightsmodel_name = 'rfwm_global_not_reshaped'

# Weights model names
weightsmodel_cv_name = 'cv_rfwm_global_not_reshaped_old_rf_params'
weightsmodel_name = 'rfwm_global_not_reshaped_old_rf_params'

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,

    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/GwSAA_old_params',
    'name_to_save': 'GwSAA_old_params',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Prepare data
    samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
    weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

    samples, actuals, weights = prep_samples_and_weights(samples, weights, SKUs=SKUs, ts=ts)
    
    # For each product (SKU) k=1,...,M
    with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
        resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=WeightedSAA(), 
                                                                 samples=samples, weights=weights[SKU], actuals=actuals[SKU], 
                                                                 **experiment_params) for SKU in SKUs)

## (b) Rolling Horizon Global Robust Weighted SAA (GwSAA-R)

...

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_global_not_reshaped'
weightsmodel_name = 'rfwm_global_not_reshaped'

# Weights model names
weightsmodel_cv_name = 'cv_rfwm_global_not_reshaped_old_rf_params'
weightsmodel_name = 'rfwm_global_not_reshaped_old_rf_params'

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,

    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/GwSAAR',
    'name_to_save_prefix': 'GwSAAR',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# For each uncertainty set specification
for e in [1,3,6,9,12]:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')
    
    # Update params
    experiment_params['name_to_save'] = experiment_params['name_to_save_prefix']+'_e'+str(e).replace('.', '')
    
    # Set path
    if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')

        # Prepare data
        samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
        weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

        samples, actuals, weights, epsilons = prep_samples_and_weights(samples, weights, e=e, SKUs=SKUs, ts=ts)

        # For each product (SKU) k=1,...,M
        with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
            resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=RobustWeightedSAA(), 
                                                                     samples=samples, weights=weights[SKU], epsilons=epsilons[SKU],
                                                                     actuals=actuals[SKU], e=e, **experiment_params) for SKU in SKUs)

## (c) Rolling Horizon Local Weighted SAA (wSAA)

...

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_local_not_reshaped'
weightsmodel_name = 'rfwm_local_not_reshaped'

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/wSAA',
    'name_to_save': 'wSAA',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Prepare data
    samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
    weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

    samples, actuals, weights = prep_samples_and_weights(samples, weights, SKUs=SKUs, ts=ts)
    
    # For each product (SKU) k=1,...,M
    with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
        resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=WeightedSAA(), 
                                                                 samples=samples[SKU], weights=weights[SKU], actuals=actuals[SKU], 
                                                                 **experiment_params) for SKU in SKUs)

## (d) Rolling Horizon Local Robust Weighted SAA (wSAA-R)

...

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_local_not_reshaped'
weightsmodel_name = 'rfwm_local_not_reshaped'

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/wSAAR',
    'name_to_save_prefix': 'wSAAR',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# For each uncertainty set specification
for e in [1,3,6,9,12]:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')
        
    # Update params
    experiment_params['name_to_save'] = experiment_params['name_to_save_prefix']+'_e'+str(e).replace('.', '')
    
    # Set path
    if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')

        # Prepare data
        samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
        weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

        samples, actuals, weights, epsilons = prep_samples_and_weights(samples, weights, e=e, SKUs=SKUs, ts=ts)

        # For each product (SKU) k=1,...,M
        with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
            resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=RobustWeightedSAA(), 
                                                                     samples=samples[SKU], weights=weights[SKU], epsilons=epsilons[SKU],
                                                                     actuals=actuals[SKU], e=e, **experiment_params) for SKU in SKUs)

## (e) Baseline model: Rolling Horizon Local Weighted SAA (SAA)

...

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/SAA',
    'name_to_save': 'SAA',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Prepare data
    samples = joblib.load(PATH_WEIGHTSMODEL+'/rfwm_local_samples_not_reshaped_tau'+str(tau)+'.joblib')
    
    samples, actuals = prep_samples_and_weights(samples, SKUs=SKUs, ts=ts)
    
    # For each product (SKU) k=1,...,M
    with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
        resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=WeightedSAA(), 
                                                                 samples=samples[SKU], actuals=actuals[SKU], 
                                                                 **experiment_params) for SKU in SKUs)

## (f) Ex-post optimal model with deterministic demand

...

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/ExPost',
    'name_to_save': 'ExPost',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Prepare data
samples = joblib.load(PATH_WEIGHTSMODEL+'/rfwm_local_samples_not_reshaped_tau'+str(0)+'.joblib')
actuals = {}
for SKU in SKUs:
    d = []
    for t in ts:
        d = d + [samples[SKU][t]['y_test'].item()]
    actuals[SKU] = np.array(d).reshape(1,len(d))

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each product (SKU) k=1,...,M
with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
    resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(SKU=SKU, wsaamodel=WeightedSAA(), actuals=actuals[SKU], **experiment_params) for SKU in SKUs)